In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import ellipk
import matplotlib.pyplot as plt
import cmath
from tqdm import tqdm
from matplotlib.animation import FuncAnimation
# plt.rcParams.update({'font.size': 20,'figure.figsize': (10.0, 6.0)})
import ipywidgets as wg
from ipywidgets import Layout
# %matplotlib widget
%matplotlib qt



In [2]:
def forceAspect(ax,aspect):
    im = ax.get_images()
    extent =  im[0].get_extent()
    ax.set_aspect(abs((extent[1]-extent[0])/(extent[3]-extent[2]))/aspect)

In [4]:
#Sweeping
num=1001
L=4e-9 #lumped L
C=250e-15 #lumped C
Z0=np.sqrt(L/C)
f0=1/(2*np.pi*np.sqrt(L*C))
V=1
I=1e-3
Rbias=V/I

omega=np.linspace(3,6,num)*2*np.pi*1e9
Cc=np.linspace(20,50,num)*1e-15  #Coupling Cap
Lbias=np.linspace(100,300,num)*1e-9 # Bias inductor

const=-3*np.ones((num))
Y=np.zeros((num,num,num),dtype=np.complex128)
Y1=np.zeros((num,num),dtype=np.complex128)
Y2=np.zeros((num),dtype=np.complex128)
Y3=np.zeros((num,num),dtype=np.complex128)
Y2eq=np.zeros((num,num),dtype=np.complex128)
S11=np.zeros((num,num,num),dtype=np.complex128)
S12=np.zeros((num,num,num),dtype=np.complex128)
BW=np.zeros((num,num))
res=np.zeros((num,num))

Q=np.zeros((num,num))
f=omega/(2*np.pi)

Y2=(1/(1j*omega*L))+(1j*omega*C) #lumped resonator Admittance

for i in range (num):
    Y1[i]=1j*omega*Cc[i]   #coupling capacitor admittance
    Y3[i]=(Rbias**(-1)+(1j*omega*Lbias[i]))**(-1) #Bias admittance        
    Y2eq[i]=Y2+Y3[i]
    
for i in tqdm(range (num), desc="demo"):
    for k in range (num):  
        Y[i,k]=((Y1[i])**(-1)+(Y2eq[k])**(-1))**(-1) #series admittance
        S11[i,k]=-Y[i,k]*Z0/(2+Y[i,k]*Z0) #Calculate S11-parameter from admittance
        S12[i,k]=2/(2+Y[i,k]*Z0) #Calculate S12-parameter from admittance
        idx= np.argwhere(np.diff(np.sign((20*np.log10(S12[i,k])) - const))).flatten()
        res[i,k] = f[(20*np.log10(S12[i,k])).argmin()]
        
        if len(idx)>1:
            BW[i,k]=np.abs(f[idx[1]]-f[idx[0]])
            Q[i,k]=res[i,k]/BW[i,k]
        else:  
            print(i,k)
            Q[i,k]=Q.max()

demo: 100%|██████████| 1001/1001 [03:36<00:00,  4.62it/s]


In [3]:
#Plotting            
fig= plt.figure()
ax = fig.add_subplot(111)
img = plt.imshow(Q,extent= [min(Cc*1e15),max(Cc*1e15),min(Lbias*1e9),max(Lbias*1e9)],cmap='viridis')
forceAspect(ax,aspect=1.0)
plt.colorbar()
plt.xlabel('$C_c$ (fF)')
plt.ylabel('$L_{bias}$ (nH)')
plt.title('2d plot of Q')

fig= plt.figure()
ax = fig.add_subplot(111)
img = plt.imshow(res,extent= [min(Cc*1e15),max(Cc*1e15),min(Lbias*1e9),max(Lbias*1e9)],cmap='viridis')
forceAspect(ax,aspect=1.0)
plt.colorbar()
plt.xlabel('$C_c$ (fF)')
plt.ylabel('$L_{bias}$ (nH)')
plt.title('2d plot of $f_0$')


# fig=plt.figure(0)
# plt.plot(f/1e9,20*np.log10(S12[100,1]),linestyle='-')
# plt.plot(f/1e9,const,linestyle='-')
# plt.ylabel("$ S_{21} (dB) $")
# plt.xlabel("$f$ (GHz)")
# plt.xlim(omega[0]/(2*np.pi*1e9),omega[-1]/(2*np.pi*1e9))
# plt.grid()
# idx= np.argwhere(np.diff(np.sign((20*np.log10(S12[100,1])) - const))).flatten()
# res = f[(20*np.log10(S12[100,1])).argmin()]
# print (res)

print(BW[100,1]*1e-6)


NameError: name 'Q' is not defined

In [4]:
#S21 Spectrum

L=[4e-9,0e-9,0e-9] #lumped L
C=180e-15 #lumped C

V=1
I=1e-3
Rbias=V/I
Z0=[]


Cc= [22*1e-15,0*1e-15,127*1e-15]
omega=np.linspace(1,6, num=10001)*2*np.pi*1e9
Lbias=[0*1e-9,0*1e-9,0*1e-9]
    
for n in range(len(Cc)):
    print('case',n+1,'--> Cc=',int(Cc[n]*1e15), 'fF, Lbias=',int(Lbias[n]*1e9),'nH, L=',L[n]*1e9,'nH')
    Z0.append(np.sqrt(L[n]/C))

for n in range(len(Cc)):
    f0=1/(2*np.pi*np.sqrt(L[n]*C))
    Y1=1j*omega*Cc[n] #coupling capacitor admittance
    Y2=(1/(1j*omega*L[n]))+(1j*omega*C) #lumped resonator Admittance
#     Y3=(Rbias**(-1)+(1j*omega*Lbias[n]))**(-1)
    Y3=0
    Y2eq=Y2+Y3

    Y=((Y1)**(-1)+(Y2eq)**(-1))**(-1) #series admittance


    S11=-Y/(2+Y*Z0[n]) #Calculate S11-parameter from admittance
    S12=(2/(2+Y*Z0[n])) #Calculate S12-parameter from admittance

    f=omega/(2*np.pi)
    print(1/Y)

    fig=plt.figure(2)
    plt.plot(f/1e9,20*np.log10(S12),linestyle='-',label='$C_c= $ %d fF, $L_{bias}= 0$ nH, $L_{res}= $ %d pH ' % (Cc[n]*1e15,L[n]*1e12))
#   , Lbias[n]*1e9
    const=-3*np.ones(10001)
    plt.plot(f/1e9,const,linestyle='-')
    plt.ylabel("$ S_{21} (dB) $")
    plt.xlabel("$f$ (GHz)")
    plt.xlim(omega[0]/(2*np.pi*1e9),omega[-1]/(2*np.pi*1e9))
    plt.grid()
#     plt.legend()
    idx = np.argwhere(np.diff(np.sign(20*np.log10(S12) - const))).flatten()
    plt.plot(f[idx]/1e9, 20*np.log10(S12[idx]), 'ro')
    BW=np.abs(f[idx[1]]-f[idx[0]])

    res = f[(20*np.log10(S12)).argmin()]  # Find the x value corresponding to the maximum y value
    print('\nf0 (case',n+1,')= ', f0*1e-9,'GHz')
    print('new f0 (case',n+1,')=' , res*1e-9,'GHz')  
    print('shift (case',n+1,')= ', abs(f0-res)*1e-6,'MHz')      
    print('BW (case',n+1,')= ', BW*1e-6, 'MHz')
    print('Q (case',n+1,')= ', res/BW)

print('\nRbias= ', Rbias*1e-3, 'k ohm')
# print('Lbias= ',Lbias*1e9, 'nH' )

case 1 --> Cc= 22 fF, Lbias= 0 nH, L= 4.0 nH
case 2 --> Cc= 0 fF, Lbias= 0 nH, L= 0.0 nH
case 3 --> Cc= 127 fF, Lbias= 0 nH, L= 0.0 nH
[0.-7208.44756917j 0.-7204.81852765j 0.-7201.19309669j ...
 0.-7778.47799921j 0.-7730.42337253j 0.-7683.06731678j]

f0 (case 1 )=  5.931354528476476 GHz
new f0 (case 1 )= 5.599 GHz
shift (case 1 )=  332.3545284764757 MHz
BW (case 1 )=  35.49999999999905 MHz
Q (case 1 )=  157.71830985915918
[nan+nanj nan+nanj nan+nanj ... nan+nanj nan+nanj nan+nanj]

f0 (case 2 )=  inf GHz
new f0 (case 2 )= 1.0 GHz
shift (case 2 )=  inf MHz
BW (case 2 )=  0.5 MHz
Q (case 2 )=  2000.0
[nan+nanj nan+nanj nan+nanj ... nan+nanj nan+nanj nan+nanj]

f0 (case 3 )=  inf GHz
new f0 (case 3 )= 1.0 GHz
shift (case 3 )=  inf MHz
BW (case 3 )=  0.5 MHz
Q (case 3 )=  2000.0

Rbias=  1.0 k ohm


/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
<ipython-input-4-d1753abe2329>:21: RuntimeWarning: divide by zero encountered in double_scalars
  f0=1/(2*np.pi*np.sqrt(L[n]*C))
<ipython-input-4-d1753abe2329>:23: RuntimeWarning: divide by zero encountered in true_divide
  Y2=(1/(1j*omega*L[n]))+(1j*omega*C) #lumped resonator Admittance
<ipython-input-4-d1753abe2329>:23: RuntimeWarning: invalid value encountered in true_divide
  Y2=(1/(1j*omega*L[n]))+(1j*omega*C) #lumped resonator Admittance
<ipython-input-4-d1753abe2329>:28: RuntimeWarning: invalid value encountered in reciprocal
  Y=((Y1)**(-1)+(Y2eq)**(-1))**(-1) #series admittance
/opt/anaconda3

In [9]:
plt.grid()